<h1> VNL Rosters + Total Stats 2021-2023</h1>

The below code creates a dataframe with the following columns:
- Jersey Number
- Player Name
- Position
- Player_ID
- Year
- Country_ID
- Nationality
- Age
- Height
- Total Points
- Avg. By Match
- Attack Points
- Efficiency
- Attack Avg. Points
- Block Points
- Block Success
- Block Avg. Points
- Serve Points
- Serve Success
- Serve Avg. Points

<h2> Importing Libraries </h2>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

<h2> This code creates a df with year, jersey number, player name and player_id </h2>

In [2]:
# List of years
years = ['2024']

# Updated mapping of years to their respective country IDs and names with corrected IDs
country_names_by_year = {
    # '2021': dict(zip([4658, 4659, 4660, 4661, 4662, 4664, 4665, 4666, 4667, 4668, 4756, 4669, 4670, 4671, 4672, 4673], ["Argentina", "Australia", "Brazil", "Bulgaria", "Canada", "France", "Germany", "Iran", "Italy", "Japan", "Netherlands", "Poland", "Russia", "Serbia", "Slovenia", "USA"])),
    # '2022': dict(zip([5136, 5137, 5138, 5139, 5140, 5218, 5141, 5142, 5143, 5144, 5145, 5146, 5147, 5149, 5150, 5151], ["Argentina", "Australia", "Brazil", "Bulgaria", "Canada", "China", "France", "Germany", "Iran", "Italy", "Japan", "Netherlands", "Poland", "Serbia", "Slovenia", "USA"])),
    # '2023': dict(zip(list(range(5818, 5834)), ["Argentina", "Brazil", "Bulgaria", "Canada", "China", "Cuba", "France", "Germany", "Iran", "Italy", "Japan", "Netherlands", "Poland", "Serbia", "Slovenia", "USA"])),
    '2024': dict(zip(list(range(6583, 6599)), ["Argentina", "Brazil", "Bulgaria", "Canada", "Cuba", "France", "Germany", "Iran", "Italy", "Japan", "Netherlands", "Poland", "Serbia", "Slovenia", "Turkey", "USA"]))
}

# Initialize an empty list to collect dataframes
df_list = []

In [3]:
# Function to remove duplicates while maintaining order
def remove_duplicates_keep_order(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

# Nested loops to iterate over each year and the respective country IDs and names
for year, countries in country_names_by_year.items():
    for country_id, country_name in countries.items():
        url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/teams/men/{country_id}/players/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')

        if table:
            html_string = str(table)
            html_io = StringIO(html_string)
            df = pd.read_html(html_io)[0]

            player_links = [a['href'] for a in soup.find_all('a', href=True) if 'players/' in a['href']]
            player_links = remove_duplicates_keep_order(player_links)
            
            player_ids = []
            for link in player_links:
                player_id = link.split('/')[-1]
                if player_id.isnumeric():
                    player_ids.append(player_id)
            
            player_ids.append('N/A') # Assuming there might be non-player links (i.e coaches!)
            player_ids += ['N/A'] * (len(df) - len(player_ids)) # Padding 'N/A'
            
            df['Player_ID'] = player_ids[:len(df)]
            df['Year'] = year
            df['Country_Name'] = country_name  # Use the name instead of ID
            
            df_list.append(df)

In [4]:
# Concatenate all dataframes into one
df_rosters = pd.concat(df_list, ignore_index=True)

In [5]:
#Checking Team USA's roster for 2024
df_rosters[(df_rosters['Country_Name'] == 'USA') & (df_rosters['Year'] == '2024')]

,No.,Player Name,Position,Player_ID,Year,Country_Name
225,3.0,Garcia,O,152845,2024,USA
226,6.0,Isaacson,S,189158,2024,USA
227,9.0,Hanes,O,170104,2024,USA
228,10.0,Dagostino,L,142445,2024,USA
229,13.0,Gasman,MB,147542,2024,USA
230,16.0,Tuaniga,S,147543,2024,USA
231,21.0,Briggs,L,168461,2024,USA
232,23.0,Kessel,OH,153472,2024,USA
233,24.0,Wildman,OH,164133,2024,USA
234,25.0,Champlin,OH,183138,2024,USA


<h2> This appends the following data to the above dataframe: Nationality, Age, Birthdate, Height, Year </h2>

In [6]:
# Function to scrape player details remains unchanged
def scrape_player_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Initialize a dictionary to hold the scraped values
    details = {'Nationality': None, 'Age': None, 'Height': None}
    
    for detail_key in details.keys():
        detail_label = soup.find(text=lambda text: text and detail_key in text)
        if detail_label and detail_label.find_next():
            details[detail_key] = detail_label.find_next().text.strip()
    
    return details

# Lists to hold the scraped details
nationalities = []
ages = []
heights = []

# Most recent year on website for age calculation
current_year = 2024

# Loop through each row in the df_rosters dataframe
for index, row in df_rosters.iterrows():
    # Skip the row if the position is COACH
    if row['Position'] == 'COACH':
        nationalities.append(None)
        ages.append(None)
        heights.append(None)
        continue
    
    # Construct the URL using the Year and Player_ID
    player_id = row['Player_ID']
    year = row['Year']
    if player_id != 'N/A':  # Check if player_id is not a coach
        url = f"https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}"
        # Scrape player details from the URL
        player_details = scrape_player_details(url)
        # Adjust the age based on the year relative to the current year (2023)
        if player_details['Age'] is not None:
            adjusted_age = int(player_details['Age']) - (current_year - int(year))
            ages.append(adjusted_age)
        else:
            ages.append(None)
    else:
        nationalities.append(None)
        ages.append(None)
        heights.append(None)
        continue
    
    nationalities.append(player_details.get('Nationality'))
    heights.append(player_details.get('Height'))

# Add the details as new columns to the df_rosters dataframe
df_rosters['Nationality'] = nationalities
df_rosters['Age'] = ages
df_rosters['Height'] = heights

df_rosters['Nationality'] = df_rosters['Nationality'].replace({'United States': 'USA'})
df_rosters.loc[df_rosters['Position'] != 'COACH', 'Height'] = df_rosters['Height'].str.replace('cm', '', regex=False)

In [7]:
#Checking Team USA's roster for 2024
df_rosters[(df_rosters['Country_Name'] == 'USA') & (df_rosters['Year'] == '2024')]

,No.,Player Name,Position,Player_ID,Year,Country_Name,Nationality,Age,Height
225,3.0,Garcia,O,152845,2024,USA,USA,25.0,200
226,6.0,Isaacson,S,189158,2024,USA,USA,25.0,187
227,9.0,Hanes,O,170104,2024,USA,USA,26.0,210
228,10.0,Dagostino,L,142445,2024,USA,USA,29.0,175
229,13.0,Gasman,MB,147542,2024,USA,USA,27.0,208
230,16.0,Tuaniga,S,147543,2024,USA,USA,27.0,191
231,21.0,Briggs,L,168461,2024,USA,USA,23.0,182
232,23.0,Kessel,OH,153472,2024,USA,USA,32.0,196
233,24.0,Wildman,OH,164133,2024,USA,USA,24.0,195
234,25.0,Champlin,OH,183138,2024,USA,USA,22.0,187


<h2> This code continues to add onto the same dataframe and places total, attacking, blocking and serving points per year per player </h2>

In [8]:
# Function to scrape player details
def scrape_player_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Initialize a dictionary to hold the scraped values
    details = {
        'Total Points': None,
        'Average by Match': None, 
        'Attack Points': None,
        'Efficiency': None,
        'Attack Avg Points': None,
        'Block Points': None,
        'Block Success': None,
        'Block Avg Points': None, 
        'Serve Points': None,
        'Serve Success': None,
        'Serve Avg Points': None
    }

    #This portion of code is to handle duplicates.
    #On the player roster, there are multiple labels for Success and Avg. Points.
    #The below ensures we're grabbing the correct value for each metric (Attack, Block, Serve)
    avg_points_labels = soup.find_all('div', class_='vbw-player-stats-head', text='Avg Points')
    success_labels = soup.find_all('div', class_='vbw-player-stats-head', text='Success')
    
    for detail_key in details.keys():
        if detail_key == 'Attack Avg Points' and avg_points_labels:
            details[detail_key] = avg_points_labels[0].find_next_sibling('div', class_='vbw-player-stats-text').text.strip()
        elif detail_key == 'Block Avg Points' and len(avg_points_labels) > 1:
            details[detail_key] = avg_points_labels[1].find_next_sibling('div', class_='vbw-player-stats-text').text.strip()
        elif detail_key == 'Serve Avg Points' and len(avg_points_labels) > 2:
            details[detail_key] = avg_points_labels[2].find_next_sibling('div', class_='vbw-player-stats-text').text.strip()
        elif detail_key == 'Block Success' and success_labels:
            details[detail_key] = success_labels[0].find_next_sibling('div', class_='vbw-player-stats-text').text.strip()
        elif detail_key == 'Serve Success' and len(success_labels) > 1:
            details[detail_key] = success_labels[1].find_next_sibling('div', class_='vbw-player-stats-text').text.strip()
        else:
            detail_label = soup.find('div', class_='vbw-player-stats-head', text=detail_key)
            if detail_label and detail_label.find_next_sibling('div', class_='vbw-player-stats-text'):
                details[detail_key] = detail_label.find_next_sibling('div', class_='vbw-player-stats-text').text.strip()
                
    return details

# Initialize lists to hold the scraped details
details_columns = {
    'Total Points': [],
    'Average by Match': [],
    'Attack Points': [],
    'Efficiency': [],
    'Attack Avg Points': [],
    'Block Points': [],
    'Block Success': [],
    'Block Avg Points': [],
    'Serve Points': [],
    'Serve Success': [],
    'Serve Avg Points': []
}

# Loop through each row in the df_rosters dataframe
for index, row in df_rosters.iterrows():
    # Skip the row if the position is COACH
    if row['Position'] == 'COACH':
        for detail in details_columns:
            details_columns[detail].append(None)
        continue
    
    # Construct the URL
    player_id = row['Player_ID']
    year = row['Year']
    if player_id != 'N/A':  # Check if player_id is not a coach
        url = f"https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}"
        # Scrape player details from the URL
        player_details = scrape_player_details(url)
    else:
        player_details = {key: None for key in details_columns.keys()}

    # Append the details to the respective lists
    for detail in details_columns:
        details_columns[detail].append(player_details[detail])

# Add the details as new columns to the df_rosters dataframe
for detail in details_columns:
    df_rosters[detail] = details_columns[detail]

#renaming 
df_rosters_24 = df_rosters

In [9]:
#Checking Team USA's roster for 2022
df_rosters[(df_rosters['Country_Name'] == 'USA') & (df_rosters['Year'] == '2024')]

,No.,Player Name,Position,Player_ID,Year,Country_Name,Nationality,Age,Height,Total Points,Average by Match,Attack Points,Efficiency,Attack Avg Points,Block Points,Block Success,Block Avg Points,Serve Points,Serve Success,Serve Avg Points
225,3.0,Garcia,O,152845,2024,USA,USA,25.0,200,19,6.33,15,39.47%,5.00,2,22.22%,0.67,2,10.53%,0.67
226,6.0,Isaacson,S,189158,2024,USA,USA,25.0,187,2,0.67,2,100%,0.67,0,0.00%,0.00,0,0.00%,0.00
227,9.0,Hanes,O,170104,2024,USA,USA,26.0,210,28,9.33,23,46.94%,7.67,4,30.77%,1.33,1,4.55%,0.33
228,10.0,Dagostino,L,142445,2024,USA,USA,29.0,175,0,0.00,0,-,-,0,-,-,0,-,-
229,13.0,Gasman,MB,147542,2024,USA,USA,27.0,208,12,4.00,8,44.44%,2.67,2,10.00%,0.67,2,8.33%,0.67
230,16.0,Tuaniga,S,147543,2024,USA,USA,27.0,191,1,0.33,1,33.33%,0.33,0,0.00%,0.00,0,0.00%,0.00
231,21.0,Briggs,L,168461,2024,USA,USA,23.0,182,0,-,0,-,-,0,-,-,0,-,-
232,23.0,Kessel,OH,153472,2024,USA,USA,32.0,196,13,4.33,11,35.48%,3.67,2,13.33%,0.67,0,0.00%,0.00
233,24.0,Wildman,OH,164133,2024,USA,USA,24.0,195,1,0.33,1,100%,0.33,0,-,-,0,0.00%,0.00
234,25.0,Champlin,OH,183138,2024,USA,USA,22.0,187,8,2.67,7,43.75%,2.33,0,0.00%,0.00,1,6.25%,0.33


In [10]:
df_rosters.tail(50)

,No.,Player Name,Position,Player_ID,Year,Country_Name,Nationality,Age,Height,Total Points,Average by Match,Attack Points,Efficiency,Attack Avg Points,Block Points,Block Success,Block Avg Points,Serve Points,Serve Success,Serve Avg Points
190,17.0,Urnaut,OH,114957,2024,Serbia,Slovenia,35.0,200,30,10.00,27,38.57%,9.00,1,7.69%,0.33,2,5.88%,0.67
191,18.0,Čebulj,OH,136918,2024,Serbia,Slovenia,32.0,202,55,18.33,46,44.23%,15.33,4,20.00%,1.33,5,10.87%,1.67
192,20.0,Mujanović,O,182325,2024,Serbia,Slovenia,19.0,206,1,0.33,0,-,-,1,50.00%,0.33,0,-,-
193,22.0,Kržič,MB,185708,2024,Serbia,Slovenia,20.0,202,0,-,0,-,-,0,-,-,0,-,-
194,NaN,Gheorghe Cretu,COACH,N/A,2024,Serbia,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None
195,2.0,Kovačević,OH,124926,2024,Slovenia,Serbia,31.0,197,19,6.33,17,45.95%,5.67,1,16.67%,0.33,1,5.88%,0.33
196,3.0,Kapur,L,136360,2024,Slovenia,Serbia,33.0,181,0,0.00,0,-,-,0,-,-,0,-,-
197,6.0,Ristić,L,188506,2024,Slovenia,Serbia,20.0,190,0,0.00,0,-,-,0,-,-,0,-,-
198,7.0,Krsmanović,MB,146579,2024,Slovenia,Serbia,33.0,205,8,2.67,7,63.64%,2.33,1,16.67%,0.33,0,0.00%,0.00
199,8.0,Ivović,OH,132022,2024,Slovenia,Serbia,33.0,194,20,6.67,19,51.35%,6.33,0,0.00%,0.00,1,3.57%,0.33


In [11]:
df_rosters_24.to_csv('df_rosters_24.csv',index=False)